# Building a Euribor Curve using non-parametric calibration with smoothing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [ ]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.rates.ibor_single_curve_smoothing_calibrator import IborSingleCurveSmoothingCalibrator

In [ ]:
valuation_date = Date(6, 10, 2001)
cal = CalendarTypes.UNITED_KINGDOM

### Instruments

In [ ]:
depo_dcc_type = DayCountTypes.ACT_360
depos = []
spot_days = 2
settle_dt = valuation_date.add_weekdays(spot_days)
depo = IborDeposit(settle_dt, "3M", 4.2/100.0, depo_dcc_type, cal_type=cal)
depos.append(depo)

fraDCCType = DayCountTypes.ACT_360
fras = []
fra = IborFRA(settle_dt.add_tenor("3M"), "3M", 4.20/100.0, fraDCCType, cal_type=cal)
fras.append(fra)

swaps = []
swapType = SwapTypes.PAY
fixedDCCType = DayCountTypes.THIRTY_E_360_ISDA
fixed_freqType = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settle_dt, "1Y", swapType, 4.20/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "2Y", swapType, 4.30/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "3Y", swapType, 4.70/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "5Y", swapType, 5.40/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "7Y", swapType, 5.70/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "10Y", swapType, 6.00/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "12Y", swapType, 6.10/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "15Y", swapType, 5.90/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "20Y", swapType, 5.60/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "25Y", swapType, 5.55/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)

Build curves for different levels of the smoothness parameter

In [ ]:
check_refit_flag = True
iborCurves = {}
reports = []   # collect fit_report DataFrames
optional_interp_params = {'sigma': 0.01}

interp_type = InterpTypes.FLAT_FWD_RATES
do_build = False

init_curve = IborSingleCurve(
    valuation_date, depos, fras, swaps,
    interp_type, check_refit_flag=False, do_build=do_build, **optional_interp_params
)
calibrator = IborSingleCurveSmoothingCalibrator(init_curve)

smooth_params = [0.01, 0.1, 0.2]
for smoothness in smooth_params:
    start = timer()

    ibor_curve, fit_report = calibrator.fit(smoothness=smoothness, report_progress=False)

    end = timer()
    print(f'smth = {smoothness} fit took {end-start:.2f} sec')

    iborCurves[smoothness] = ibor_curve
    reports.append(fit_report)

# build the combined report once at the end
full_report = pd.concat(reports, ignore_index=True)


### Look at the fit reports

In [ ]:
# Print DataFrames in full
pd.set_option('display.max_rows', None,
                    'display.max_columns', None,
                    'display.float_format', lambda x:f'{x:.4f}'
                    )

In [ ]:
# examine the last fit_report
fit_report.head()

In [ ]:
# combine fit reports for different smothness parameters into one for ease of comparison
fr2 = full_report[['tgt_label','value_in_bps', 'smoothness']].set_index(['tgt_label', 'smoothness']).unstack(level=1).droplevel(0,axis=1)
fr2 = fr2.join(fit_report[['tgt_label','type','start_dt',	'maturity_dt']].set_index('tgt_label'))
fr2

### Curve Shape Analysis

First check instantaneous (ON) forward rates and see how they are affected by the smoothness parameter

In [ ]:
# grid of year fractions
years = np.linspace(1.0/365.0, 25.0, 25*365)

# build plot dates (list, not vectorized numpy)
plotDates = [settle_dt.add_years(float(y)) for y in years]

# If you're in Jupyter, keep this. Otherwise, remove the next line.
# %matplotlib widget

plt.figure(figsize=(8, 5))
for smoothness in smooth_params:
    ibor_curve = iborCurves[smoothness]
    iborCurveFwdRate = ibor_curve.fwd(plotDates)   # expects list of Dates
    plt.plot(years, iborCurveFwdRate * 100.0, label=f"smth={smoothness}")

# Use enum name if available, else str()
title_interp = getattr(interp_type, "name", str(interp_type))
plt.title(f"ON fwd rates ({title_interp})")
plt.xlabel("Years")
plt.ylabel("Rate (%)")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

Observe smoothness at work for zero rates (yields)

In [ ]:
years = np.linspace(1.0/365.0, 25.0, 25*365)
plotDates = [settle_dt.add_years(float(y)) for y in years]

# If using Jupyter and want interactive zoom:
# %matplotlib widget

plt.figure(figsize=(8,5))
for smoothness in smooth_params:
    ibor_curve = iborCurves[smoothness]
    iborCurveZeroRate = ibor_curve.zero_rate(plotDates)
    plt.plot(years, iborCurveZeroRate*100.0, label=f"smth={smoothness}")

plt.xlabel("Years")
plt.ylabel("Zero Rate (%)")
plt.title("Yields (zero rates)")
plt.legend(loc="best")
plt.tight_layout()
plt.show()



Compare output swap rates vs targets

In [ ]:
# %matplotlib widget   # keep only if you're in Jupyter

mat_years = np.linspace(1.0, 25.0, 25, endpoint=True)
mat_dates = [settle_dt.add_years(float(y)) for y in mat_years]  # list, not vectorized

# Market targets from your swaps
tgt_dates = [s.fixed_leg.maturity_dt for s in swaps]
tgt_years = [(d - valuation_date) / G_DAYS_IN_YEARS for d in tgt_dates]
tgt_rates = np.array([s.fixed_leg.cpn for s in swaps])

# Use the first swap as a template for conventions/effective date
tmpl = swaps[0]
eff_dt = tmpl.effective_dt
fix_freq = tmpl.fixed_leg.freq_type
fix_dc = tmpl.fixed_leg.dc_type

plt.figure(figsize=(8, 5))
plt.plot(tgt_years, tgt_rates * 100.0, 'o', label="swap rate targets")

for smoothness in smooth_params:
    ibor_curve = iborCurves[smoothness]
    iborCurveSwapRates = ibor_curve.swap_rate(eff_dt, mat_dates, fix_freq, fix_dc)
    plt.plot(mat_years, iborCurveSwapRates * 100.0, '.', label=f"smth={smoothness}")

title_interp = getattr(interp_type, "name", str(interp_type))
plt.title(f"Swap rates vs market quotes ({title_interp})")
plt.xlabel("Years")
plt.ylabel("Rate (%)")
plt.legend(loc="best")
plt.tight_layout()
plt.show()
